In [ ]:
# UNCOMMENT THE FOLLOWING IF YOU ARE USING KAGGLE
# !pip install segmentation-models

In [ ]:
# UNCOMMENT THE FOLLOWING IF YOU ARE USING KAGGLE
# import os
# os.environ['SM_FRAMEWORK'] = 'tf.keras'

from PIL import Image
import joblib
import numpy as np
import segmentation_models as sm
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import keras.backend as K
import tensorflow as tf

if not hasattr(K, 'sigmoid'):
    K.sigmoid = tf.nn.sigmoid

# UNCOMMENT THE FOLLOWING IF YOU ARE USING KAGGLE
# sm.set_framework('tf.keras')
# print("Framework set to:", sm.framework())

tf.random.set_seed(42)

In [ ]:
NUM_CLASSES = 6
IMAGE_PATCH_SIZE = 256
NUM_CHANNELS = 3

In [ ]:
metrics = ['accuracy', sm.metrics.IOUScore(threshold=0.5)]

<img src="https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png" alt="U-Net Architecture" style="width: 70%; height: auto;">

### Divergence from the original U-Net (2015) a little bit

- We're adding **dropout** to avoid overfitting; however, it doesn't exist in the original U-Net (2015).
	- Dropout rates increase deeper in the encoder: 0.1 → 0.3, because deeper layers tend to overfit more.

- We're `padding='same'`; however, the original U-Net used 'valid' padding but also performed manual cropping before concatenation to make dimensions match.

- Reduced the number of filters on each step, as the original architecture needs a very powerful GPU (more powerful than Kaggle's)

In [ ]:
# custom Conv-BN-ReLU-Dropout block
def ConvBNDropout(x, filters, dropout_rate=0.1):
    x = tf.keras.layers.Conv2D(filters, kernel_size=3, padding='same', 
                               kernel_initializer='he_normal', use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    return x

# custom Transposed Convolution block 
def Upsample(filters, x):
    return tf.keras.layers.Conv2DTranspose(filters, kernel_size=2, strides=2, 
                                  padding='same', kernel_initializer='he_normal')(x)


def build_unet(input_shape=(IMAGE_PATCH_SIZE, IMAGE_PATCH_SIZE, NUM_CHANNELS), 
               dropout_rate=0.1, num_classes=NUM_CLASSES):
    
    input = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    c1 = ConvBNDropout(input, 32, dropout_rate)
    c1 = ConvBNDropout(c1, 32, dropout_rate)
    p1 = tf.keras.layers.MaxPooling2D()(c1)

    c2 = ConvBNDropout(p1, 64, dropout_rate)
    c2 = ConvBNDropout(c2, 64, dropout_rate)
    p2 = tf.keras.layers.MaxPooling2D()(c2)

    c3 = ConvBNDropout(p2, 128, dropout_rate)
    c3 = ConvBNDropout(c3, 128, dropout_rate)
    p3 = tf.keras.layers.MaxPooling2D()(c3)

    c4 = ConvBNDropout(p3, 256, dropout_rate)
    c4 = ConvBNDropout(c4, 256, dropout_rate)
    p4 = tf.keras.layers.MaxPooling2D()(c4)

    # Bottleneck
    c5 = ConvBNDropout(p4, 512, dropout_rate)
    c5 = ConvBNDropout(c5, 512, dropout_rate)

    # Decoder
    u6 = Upsample(256, c5)
    u6 = tf.keras.layers.Concatenate()([u6, c4])
    c6 = ConvBNDropout(u6, 256, dropout_rate)
    c6 = ConvBNDropout(c6, 256, dropout_rate)

    u7 = Upsample(128, c6)
    u7 = tf.keras.layers.Concatenate()([u7, c3])
    c7 = ConvBNDropout(u7, 128, dropout_rate)
    c7 = ConvBNDropout(c7, 128, dropout_rate)

    u8 = Upsample(64, c7)
    u8 = tf.keras.layers.Concatenate()([u8, c2])
    c8 = ConvBNDropout(u8, 64, dropout_rate)
    c8 = ConvBNDropout(c8, 64, dropout_rate)

    u9 = Upsample(32, c8)
    u9 = tf.keras.layers.Concatenate()([u9, c1])
    c9 = ConvBNDropout(u9, 32, dropout_rate)
    c9 = ConvBNDropout(c9, 32, dropout_rate)

	# each pixel has a vector of probabilities over all classes; that's why we have num_classes feature maps
    output = tf.keras.layers.Conv2D(num_classes, kernel_size=1, activation='softmax')(c9)

    model = tf.keras.Model(inputs=[input], outputs=[output])
    return model

In [ ]:
model = build_unet()

In [ ]:
X_train, X_test, y_train, y_test = joblib.load(r'..\data\dataset.joblib')

In [ ]:
tf.random.set_seed(42)

early_stopping = tf.keras.callbacks.EarlyStopping(restore_best_weights=True, 
                                                  patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                                 patience=3)

model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=metrics)

##### Plot model diagnostics

In [ ]:
%matplotlib inline

In [ ]:
# from utils.py in the root dir
class PlotDiagnostics(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.epoch_count = 0
        self.x = []

        self.losses = []
        self.val_losses = []

        self.iou_score = []
        self.val_iou_score = []

        self.accuracy = []
        self.val_accuracy = []

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.epoch_count)

        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

        self.iou_score.append(logs.get('iou_score'))
        self.val_iou_score.append(logs.get('val_iou_score'))

        self.accuracy.append(logs.get('accuracy'))
        self.val_accuracy.append(logs.get('val_accuracy'))

        self.epoch_count += 1

        clear_output(wait=True)
        plt.figure(figsize=(18, 5))
        f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5), sharex=True)

        # Plot Loss
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.set_title("Loss")
        ax1.set_yscale('log')
        ax1.legend()

        # Plot IoU (Jaccard Index)
        ax2.plot(self.x, self.iou_score, label="iou_score")
        ax2.plot(self.x, self.val_iou_score, label="val_iou_score")
        ax2.set_title("IoU Score")
        # ax2.set_yscale('log')
        ax2.legend()

        # Plot Accuracy
        ax3.plot(self.x, self.accuracy, label="accuracy")
        ax3.plot(self.x, self.val_accuracy, label="val_accuracy")
        ax3.set_title("Accuracy")
        ax3.legend()

        plt.tight_layout()
        plt.show()

In [ ]:
plot_diagnostics = PlotDiagnostics()

In [ ]:
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), 
                    callbacks=[plot_diagnostics, early_stopping, reduce_lr])

### Comparing prediction results 

In [ ]:
X_test.shape

In [ ]:
print(y_test.shape)
y_test = np.argmax(y_test, axis=-1)
print(y_test.shape)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred.shape)

y_pred = np.argmax(y_pred, axis=-1)
print(y_pred.shape)

`matplotlib.pyplot.imshow()` assigns colors automatically based on the values. So even if y_test and y_pred have the same values, the colors may differ. It doesn’t know which color corresponds to which class unless you force it to using a colormap.

With the fixed `ListedColormap`, both the ground truth and predicted images will show the same colors for the same classes, making visual comparison meaningful.

In [ ]:
color_list = [
    '#E2A929',  # Class 0
    '#8429F6',  # Class 1
    '#6EC1E4',  # Class 2
    '#3C1098',  # Class 3
    '#FEDD3A',  # Class 4
    '#9B9B9B'   # Class 5
]

cmap = ListedColormap(color_list)

In [ ]:
for i in range(0, 9, 4): 
    original_image = X_test[i] 
    ground_truth_image = y_test[i] 
    predicted_image = y_pred[i] 

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 3, 1) 
    plt.title("Original Image") 
    plt.imshow(original_image) 
    plt.axis('off')

    plt.subplot(1, 3, 2) 
    plt.title("Ground Truth") 
    plt.imshow(ground_truth_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)
    plt.axis('off')

    plt.subplot(1, 3, 3) 
    plt.title("Prediction") 
    plt.imshow(predicted_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)
    plt.axis('off')

    plt.tight_layout() 
    plt.show()

In [ ]:
model.save("satellite_segmentation_full.keras")

### Create segmentation model with pretrained encoder

In [ ]:
X_train, X_test, y_train, y_test = joblib.load(r'../data/dataset.joblib')

In [ ]:
model = sm.Unet('efficientnetb4', input_shape=(IMAGE_PATCH_SIZE, IMAGE_PATCH_SIZE, NUM_CHANNELS), 
                classes=NUM_CLASSES, activation='softmax', encoder_weights='imagenet')

In [ ]:
tf.random.set_seed(42)

early_stopping = tf.keras.callbacks.EarlyStopping(restore_best_weights=True, 
                                                  patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, 
                                                 patience=3)

model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
history = model.fit(X_train, y_train, epochs=100, 
                    validation_data=(X_test, y_test), 
                    callbacks=[plot_diagnostics, early_stopping, reduce_lr])

It's clear that the new pretrained model (accuracy of "0.89" and IoU of "0.68") is better than the model I've created from scratch (accuracy of "0.84" and IoU of "0.55"). So, let's use this pretrained model instead when deploying to hugging face.

In [ ]:
y_test = np.argmax(y_test, axis=-1)
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
color_list = [
    '#E2A929',  # Class 0
    '#8429F6',  # Class 1
    '#6EC1E4',  # Class 2
    '#3C1098',  # Class 3
    '#FEDD3A',  # Class 4
    '#9B9B9B'   # Class 5
]
cmap = ListedColormap(color_list)

for i in range(0, 9, 4): 
    original_image = X_test[i] 
    ground_truth_image = y_test[i] 
    predicted_image = y_pred[i] 

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 3, 1) 
    plt.title("Original Image") 
    plt.imshow(original_image) 
    plt.axis('off')

    plt.subplot(1, 3, 2) 
    plt.title("Ground Truth") 
    plt.imshow(ground_truth_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)
    plt.axis('off')

    plt.subplot(1, 3, 3) 
    plt.title("Prediction") 
    plt.imshow(predicted_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)
    plt.axis('off')

    plt.tight_layout() 
    plt.show()

model.save("satellite_segmentation_model_pretraining.keras")

### Performing prediction using an image from Google Map

In [ ]:
model = tf.keras.models.load_model(
    '../models/satellite_segmentation_model_pretraining.keras',
    compile=False)

model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=metrics)

In [ ]:
color_list = [
    '#E2A929',  # Class 0
    '#8429F6',  # Class 1
    '#6EC1E4',  # Class 2
    '#3C1098',  # Class 3
    '#FEDD3A',  # Class 4
    '#9B9B9B'   # Class 5
]
cmap = ListedColormap(color_list)

In [ ]:
# https://project.inria.fr/aerialimagelabeling/
image = Image.open(r"..\visulas\test\test2_InriaAerialImageLabeling.jpg")
image = image.resize((IMAGE_PATCH_SIZE, IMAGE_PATCH_SIZE))
image = np.array(image)[:, :, :3] # keep only RGB; drop alpha channel
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')  
image = np.expand_dims(image, axis=0) # batch of size 1
image = image / 255.0

predicted_image = model.predict(image)
predicted_image = np.argmax(predicted_image, axis=-1)
predicted_image = predicted_image[0, ...]
plt.subplot(1, 2, 2)
plt.imshow(predicted_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)   
plt.axis('off') 

plt.tight_layout()
plt.show()

In [ ]:
# https://phys.org/news/2012-08-satellite-view-house.html
image = Image.open(r"..\visulas\test\test1_GoogleMaps.png")
image = image.resize((IMAGE_PATCH_SIZE, IMAGE_PATCH_SIZE))
image = np.array(image)[:, :, :3] # keep only RGB; drop alpha channel
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')  
image = np.expand_dims(image, axis=0) # batch of size 1
image = image / 255.0

predicted_image = model.predict(image)
predicted_image = np.argmax(predicted_image, axis=-1)
predicted_image = predicted_image[0, ...]
plt.subplot(1, 2, 2)
plt.imshow(predicted_image, cmap=cmap, vmin=0, vmax=len(color_list) - 1)   
plt.axis('off') 

plt.tight_layout()
plt.show()